In [119]:
import requests
import os
import json

from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

from groq import Groq

In [120]:
def get_documents():
    docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()
    
    documents = []
    
    for course in documents_raw:
        course_name = course['course']
    
        for doc in course['documents']:
            doc['course'] = course_name
            documents.append(doc)

    return documents

# get_documents()

In [121]:
# Get ElasticSearch client

# First we delete the existent index to create a new one
if (es_client.indices.exists(index="course_questions")):
    !curl -X DELETE 'http://localhost:9200/course_questions'

# Health check
es_client = Elasticsearch("http://localhost:9200")
es_client.info()
    
# Create index. 
# We are interested in Mappings that define the structure of the documents within an index. Each field 
# in the documents can have specific data types and properties. We use the "keyword" type for a field 
# like "course" when we want to filter documents based on the course name or run aggregations to count
# the number of documents per course.
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_questions"
es_client.indices.create(index=index_name, body=index_settings)

# I traverse the list of documents and add into the index
for doc in tqdm(get_documents()):
    es_client.index(index=index_name, document=doc)

{"acknowledged":true}

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:24<00:00, 38.88it/s]


In [123]:
def elastic_search(query: str) -> list:
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])
    #result_docs = [hit['_source'] for hit in response['hits']['hits']]
    
    return result_docs

In [124]:
def build_prompt(query: str, search_results: list) -> str:
    prompt_template = """
        You're a course teaching assistant. You will answer the QUESTION using information from CONTEXT only.
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT doesn't contains the answer, output NONE.
    
        QUESTION: {question}
    
        CONTEXT:
        {context}
    """.strip()
    
    context = ""

    for doc in search_results:
        context += f'section: {doc["section"]}\nquestion: {doc["question"]}\nanswer:: {doc["text"]}\n\n'
        
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [125]:
# Get the client of Groq
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [126]:
def llm(prompt: str, model: str = "llama3-8b-8192") -> str | None:
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )

    return response.choices[0].message.content

In [127]:
def rag(query: str, model: str = "llama3-8b-8192") -> str | None:
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt, model)

In [128]:
rag("How do I run Kafka?")

'To run Kafka, you can use the command:\n\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n\nThis command was extracted from the context in the section "Module 6: streaming with kafka" under the question "Java Kafka: How to run producer/consumer/kstreams/etc in terminal".'

In [ ]:
# *************************** HOMEWORK **********************************************

In [150]:
def elastic_search_homework(query: str, size: int = 5, course: str = "data-engineering-zoomcamp") -> list:
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])
    #result_docs = [hit['_source'] for hit in response['hits']['hits']]
    
    return result_docs, response

In [145]:
query_homework = "How do I execute a command in a running docker container?"
search_results_homework, response = elastic_search_homework(query_homework)

In [146]:
response

ObjectApiResponse({'took': 14, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 390, 'relation': 'eq'}, 'max_score': 75.54128, 'hits': [{'_index': 'course_questions', '_id': 'T66HapABaFiur6FkRyK_', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n

In [151]:
query_homework = "How do I execute a command in a running docker container?"
search_results_homework, response = elastic_search_homework(query_homework, 3, "machine-learning-zoomcamp")

In [152]:
search_results_homework

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [154]:
def build_prompt_homework(query: str, search_results: list) -> str:
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
    
        QUESTION: {question}
    
        CONTEXT:
        {context}
    """.strip()
    
    context = ""

    for doc in search_results:
        context += f'Q: {doc["question"]}\nA: {doc["text"]}\n\n'
        
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [170]:
query_homework = "How do I execute a command in a running docker container?"
prompt_homework = build_prompt_homework(query_homework, search_results_homework)
len(prompt_homework)

1502

In [171]:
import tiktoken

In [172]:
encoding = tiktoken.encoding_for_model("gpt-4o")
my_encoding = encoding.encode(prompt_homework)
len(my_encoding)

328